In [1]:
from __future__ import print_function, division
from heartnet_v1 import heartnet, reshape_folds
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
set_session(tf.Session(config=config))
import numpy as np
np.random.seed(1)
from tensorflow import set_random_seed
set_random_seed(1)
from keras.utils import to_categorical
from keras.optimizers import Adam, SGD
from keras.callbacks import CSVLogger, ModelCheckpoint, TensorBoard, Callback
from keras.utils import plot_model
from keras.layers import Dense, Dropout
from keras.models import Model
from sklearn.metrics import recall_score, confusion_matrix
import pandas as pd
import os
import tables
from datetime import datetime
import argparse

def heartnet_transfer(load_path='/media/taufiq/Data/heart_sound/interspeech_compare/weights.0148-0.8902.hdf5',lr=0.0012843784,lr_decay=0.0001132885,num_dense=20,trainable=False,dropout_rate=0.):
    model = heartnet(load_path=load_path,FIR_train=False,trainable=trainable)
    x = model.layers[-4].output
    x = Dense(num_dense,activation='relu') (x)
    x = Dropout(rate=dropout_rate,seed=1) (x)
    output = Dense(3,activation='softmax')(x)
    model = Model(inputs=model.input,outputs=output)
    sgd = SGD(lr=lr)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

class log_UAR(Callback):

    def __init__(self, x_val, y_val, val_parts, res_thresh):
        self.x_val = x_val
        self.y_val = y_val
        self.val_parts = val_parts
        self.res_thresh = res_thresh

    def on_epoch_end(self, epoch, logs):
        if logs is not None:
            y_pred = self.model.predict(self.x_val, verbose=0)
            y_pred = np.argmax(y_pred, axis=-1)
            self.y_val_ = np.transpose(np.argmax(self.y_val, axis=-1))
            true = []
            pred = []
            start_idx = 0
            for s in self.val_parts:

                if not s:  ## for e00032 in validation0 there was no cardiac cycle
                    continue
                # ~ print "part {} start {} stop {}".format(s,start_idx,start_idx+int(s)-1)

                temp_ = self.y_val_[start_idx:start_idx + int(s) - 1]
                temp = y_pred[start_idx:start_idx + int(s) - 1]

                if (sum(temp == 0) > sum(temp == 1)) and (sum(temp == 0) > sum(temp == 2)):
                    pred.append(0)
                elif (sum(temp == 2) > sum(temp == 1)) and (sum(temp == 2) > sum(temp == 0)):
                    pred.append(2)
                else:
                    pred.append(1)

                if (sum(temp_ == 0) > sum(temp_ == 1)) and (sum(temp_ == 0) > sum(temp_ == 2)):
                    true.append(0)
                elif (sum(temp_ == 2) > sum(temp_ == 1)) and (sum(temp_ == 2) > sum(temp_ == 0)):
                    true.append(2)
                else:
                    true.append(1)

                start_idx = start_idx + int(s)

            score = recall_score(y_pred=pred, y_true=true, average='macro')
            logs['UAR'] = np.array(score)


if __name__ == '__main__':

    fold_dir = '/media/taufiq/Data/heart_sound/interspeech_compare/feature/segmented_noFIR/'
    foldname = 'comParE'
    model_dir = '/media/taufiq/Data/heart_sound/models/'
    log_name = foldname + ' ' + str(datetime.now())
    checkpoint_name = model_dir + log_name + "/" + 'weights.{epoch:04d}-{val_acc:.4f}.hdf5'
    if not os.path.exists(model_dir + log_name):
        os.makedirs(model_dir + log_name)
    log_dir = '/media/taufiq/Data/heart_sound/interspeech_compare/logs/'

    ##### Load Model ######

    load_path='/media/taufiq/Data/heart_sound/interspeech_compare/weights.0148-0.8902.hdf5'
    lr = 0.00001
    num_dense = 34
    epochs = 1
    batch_size = 128
    dropout_rate = 0.
    trainable = True
    res_thresh = .5
    model = heartnet_transfer(load_path=load_path,lr=lr,num_dense=num_dense,trainable=trainable,dropout_rate=dropout_rate)
    plot_model(model,"model.png",show_layer_names=True,show_shapes=True)

    ###### Load Data ######

    feat = tables.open_file(fold_dir + foldname + '.mat')
    x_train = feat.root.trainX[:]
    y_train = feat.root.trainY[0, :]
    x_val = feat.root.valX[:]
    y_val = feat.root.valY[0, :]
    train_parts = feat.root.train_parts[:]
    val_parts = feat.root.val_parts[0, :]

    ############### Reshaping ############

    x_train, y_train, x_val, y_val = reshape_folds(x_train, x_val, y_train, y_val)
    y_train = to_categorical(y_train,num_classes=3)
    y_val = to_categorical(y_val,num_classes=3)

    ### Callbacks ###
    csv_logger = CSVLogger(log_dir + log_name + '/training.csv')
    modelcheckpnt = ModelCheckpoint(filepath=checkpoint_name,
                                    monitor='val_acc', save_best_only=True, mode='max')
    tensbd = TensorBoard(log_dir=log_dir + log_name,
                         batch_size=batch_size,
                         histogram_freq=100,
                         # embeddings_freq=99,
                         # embeddings_layer_names=embedding_layer_names,
                         # embeddings_data=x_val,
                         # embeddings_metadata=metadata_file,
                         write_images=False)

    ### Train ###
    model.fit(x_train,y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=2,
              shuffle=True,
              callbacks=[modelcheckpnt,
                         log_UAR(x_val, y_val, val_parts, res_thresh),
                         tensbd, csv_logger],
              validation_data=(x_val,y_val))

    ###### Results #####

    y_pred = model.predict(x_val, verbose=0)
    y_pred = np.argmax(y_pred,axis=-1)
    y_val = np.transpose(np.argmax(y_val,axis=-1))
    true = []
    pred = []
    start_idx = 0
    for s in val_parts:

        if not s:  ## for e00032 in validation0 there was no cardiac cycle
            continue
        # ~ print "part {} start {} stop {}".format(s,start_idx,start_idx+int(s)-1)

        temp_ = y_val[start_idx:start_idx + int(s) - 1]
        temp = y_pred[start_idx:start_idx + int(s) - 1]

        if (sum(temp==0) > sum(temp==1)) and  (sum(temp==0) > sum(temp==2)):
            pred.append(0)
        elif (sum(temp==2) > sum(temp==1)) and  (sum(temp==2) > sum(temp==0)):
            pred.append(2)
        else:
            pred.append(1)

        if (sum(temp_ == 0) > sum(temp_ == 1)) and (sum(temp_ == 0) > sum(temp_ == 2)):
            true.append(0)
        elif (sum(temp_ == 2) > sum(temp_ == 1)) and (sum(temp_ == 2) > sum(temp_ == 0)):
            true.append(2)
        else:
            true.append(1)

        start_idx = start_idx + int(s)
    score = recall_score(y_pred=pred, y_true=true, average='macro')
    print(score)
    print(confusion_matrix(y_true=true,y_pred=pred))


Using TensorFlow backend.


(17061, 2500, 1)
(17061, 1)
(5872, 2500, 1)
(5872, 1)
Train on 17061 samples, validate on 5872 samples
Epoch 1/100
 - 16s - loss: 1.5462 - acc: 0.3087 - val_loss: 1.1749 - val_acc: 0.3595
Epoch 2/100


IndexError: invalid index to scalar variable.

In [ ]:
    y_pred = model.predict(x_val, verbose=0)
    print(y_pred)

In [ ]:
np.max(y_pred[:,-1])

In [ ]:
y = np.argmax(y_pred)
sum()